In [0]:
! pip install wosplus openpyxl >/dev/null

In [3]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx               = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x
datos1.csv              = 11CyLRZZwVbgw6YAC-igRJ3mrkIwk0aaiXnd-EOofYTI
oadoi_udea.json         = 1s508qokcIkz-i1Y4Ukbdht-jWuSVngON

Writing drive.cfg


In [0]:
import wosplus as wp
files=wp.wosplus('drive.cfg')

In [0]:
import pandas as pd

In [0]:
df=files.read_drive_json('oadoi_udea.json')

In [0]:
dfin=df[df.best_oa_location.isnull()].reset_index(drop=True)
dfinn=df[~df.best_oa_location.isnull()].reset_index(drop=True)
dfinn['url_for_pdf']=dfinn.best_oa_location.apply(lambda x: x.get('url_for_pdf'))
dfin['url_for_pdf']=None
dff=dfinn.append(dfin).reset_index(drop=True)

In [11]:
dff[:2]

,best_oa_location,data_standard,doi,doi_url,genre,is_oa,journal_is_in_doaj,journal_is_oa,journal_issns,journal_name,oa_locations,published_date,publisher,title,updated,year,z_authors,url_for_pdf
0,"{'pmh_id': None, 'updated': '2017-09-02T17:22:...",2,10.1001/archinte.166.1.95,https://doi.org/10.1001/archinte.166.1.95,journal-article,True,False,False,0003-9926,Archives of Internal Medicine,"[{'pmh_id': None, 'updated': '2017-09-02T17:22...",2006-01-09,American Medical Association (AMA),Etiologic Diagnosis of Chronic Osteomyelitis,2018-06-15T03:19:43.547479,2006.0,"[{'given': 'Andres F.', 'family': 'Zuluaga'}]",http://archinte.jamanetwork.com/data/journals/...
1,"{'pmh_id': None, 'updated': '2017-09-27T16:19:...",2,10.1001/archneur.55.6.857,https://doi.org/10.1001/archneur.55.6.857,journal-article,True,False,False,0003-9942,Archives of Neurology,"[{'pmh_id': None, 'updated': '2017-09-27T16:19...",1998-06-01,American Medical Association (AMA),Patients With Epilepsy Who Die Suddenly Have C...,2018-06-19T02:56:53.795724,1998.0,"[{'given': 'Benjamin H.', 'family': 'Natelson'...",http://archneur.jamanetwork.com/data/journals/...
